In [3]:
import brightway2 as bw
import bentso
from bentso.iterators import COUNTRIES

In [2]:
bw.projects.set_current("SETAC 2019")

In [3]:
if "ecoinvent 3.5 cutoff" not in bw.databases:
    bw.bw2setup()
    ei = bw.SingleOutputEcospold2Importer("/Users/cmutel/Documents/LCA/Ecoinvent/3.5/cutoff/datasets", "ecoinvent 3.5 cutoff")
    ei.apply_strategies()
    ei.write_database()

# Mapping ENTSO-E labels to ecoinvent

List all ecoinvent activities that have the unit `kilowatt hour`:

In [9]:
sorted({x['name'] for x in bw.Database("ecoinvent 3.5 cutoff") if x['unit'] == 'kilowatt hour'})

['biogas, burned in micro gas turbine 100kWe',
 'biogas, burned in polymer electrolyte membrane fuel cell 2kWe, future',
 'biogas, burned in solid oxide fuel cell 125kWe, future',
 'biogas, burned in solid oxide fuel cell, with micro gas turbine, 180kWe, future',
 'burnt shale production',
 'cane sugar production with ethanol by-product',
 'electricity production, coal, aluminium industry',
 'electricity production, deep geothermal',
 'electricity production, hard coal',
 'electricity production, hard coal, at coal mine power plant',
 'electricity production, hard coal, conventional',
 'electricity production, hard coal, supercritical',
 'electricity production, high voltage, for Swiss Federal Railways',
 'electricity production, hydro, aluminium industry',
 'electricity production, hydro, pumped storage',
 'electricity production, hydro, reservoir, alpine region',
 'electricity production, hydro, reservoir, alpine region, label-certified',
 'electricity production, hydro, reservoir, n

Some labels are easy, but a few have no obvious analogues:

* 'Fossil Coal-derived gas'
* 'Other renewable'
* 'Other'

Check to see how important they are in the ENTSO data.

You can set the key to a different value if you don't have the data cached already.

In [2]:
c = bentso.CachingDataClient(key="cache-only")

Using data directory /Users/cmutel/Library/Application Support/bentso


In [24]:
missing = ['Other renewable', 'Other']

In [25]:
for country in COUNTRIES:
    df = c.get_generation(country, 2017)
    if df is None:
        continue
    for label in missing:
        if label in df.columns:
            total = df[label].sum()
            if total > 0:
                print(country, label, total, total / df.sum().sum())

AT Other 770880.0 0.0032773083017291335
BE Other 5080873.0 0.06245732457602635
Value not in cache; returning nothing.
Value not in cache; returning nothing.
CZ Other renewable 2482996.0 0.030779251324929282
CZ Other 975209.0 0.012088703689145275
EE Other renewable 54694.0 0.004970315238886197
FI Other renewable 354906.0 0.005735598610298779
FI Other 1132735.0 0.018306011427918344
DE Other renewable 6365306.0 0.0029486705610072377
DE Other 165030109.0 0.0764487086855079
HU Other renewable 401251.0 0.0033896873540959706
HU Other 2970384.0 0.025093203709421297
IE Other 26671.0 0.0007093943672262622
Value not in cache; returning nothing.
IT Other 81423613.0 0.32047333882073326
LV Other 784921.0 0.10756488871189525
Value not in cache; returning nothing.
Value not in cache; returning nothing.
NL Other 6864.0 3.3435133670610246e-05
Value not in cache; returning nothing.
NO Other 380779.0 0.0025892372053025635
PT Other 318916.0 0.005834667717159087
RS Other 305419.0 0.008984141270862745
SK Oth

Traditionally, `other renewable` is wind, solar, hydro, biofuels, tidal, and geothermal - but we have all of these covered. Therefore, we will set `other renewable` to the country-specific mix of these renewables, as we don't have any better data.

`other` is a tougher problem. Italy has an astonishingly high fraction, 0.32. We can compare the ENTSO numbers with more [details accounts from Terna, the Italian ISO](http://download.terna.it/terna/0000/1112/76.PDF) for 2016:

| ENTSO Label | ENTSO value (GWh) | Terna value (GWh) | Note |
| --- | --- | --- | --- |
| Biomass                         |  2.4 | 4.0 |
| Fossil Coal-derived gas         |  1.9 | 2.7 |
| Fossil Gas                      | 61.0 | 123 |
| Fossil Hard coal                | 10.0 | 32.4 |
| Fossil Oil                      |  1.2 | 3.8 |
| Geothermal                      |  5.8 | 5.8 |
| Hydro Pumped Storage            |  3.1 | 42.4 | All hydro |
| Hydro Run-of-river and poundage | 31.0 |  |
| Hydro Water Reservoir           |  6.3 |  |
| Other                           | 92.2 |  |
| Solar                           | 18.0 | 22.1 |
| Waste                           |   .3 | 15.5 |
| Wind Onshore                    | 17.1 | 17.7 |
| --- | --- | --- | --- |

It looks like the best we can do (without doing very detailed analysis for each country with `other`) is to assign the complete mix to `other`.

In [27]:
generic_mapping = {
    'Solar': [
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si',
        'electricity production, photovoltaic, 3kWp flat-roof installation, single-Si',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CIS, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CdTe, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted, label-certified',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted, label-certified',
        'electricity production, photovoltaic, 570kWp open ground installation, multi-Si',
        'electricity production, solar thermal parabolic trough, 50 MW',
        'electricity production, solar tower power plant, 20 MW',        
    ],
    'Wind Onshore': [
        'electricity production, wind, 1-3MW turbine, onshore',
        'electricity production, wind, 1-3MW turbine, onshore, label-certified',
        'electricity production, wind, 2.3MW turbine, precast concrete tower, onshore',
        'electricity production, wind, <1MW turbine, onshore',
        'electricity production, wind, <1MW turbine, onshore, label-certified',
        'electricity production, wind, >3MW turbine, onshore',
     ],
    'Wind Offshore': [
        'electricity production, wind, 1-3MW turbine, offshore'
    ],
    'Hydro Water Reservoir': [
        'electricity production, hydro, reservoir, alpine region',
        'electricity production, hydro, reservoir, alpine region, label-certified',
        'electricity production, hydro, reservoir, non-alpine region',
        'electricity production, hydro, reservoir, tropical region',
        'electricity production, hydro, reservoir, alpine region, label-certified',
    ],
    'Hydro Run-of-river and poundage': [
        'electricity production, hydro, run-of-river',
        'electricity production, hydro, run-of-river, label-certified',
    ],
    'Waste': [
        'electricity, from municipal waste incineration to generic market for electricity, medium voltage'
    ],
    'Marine': [
        'Production of electricity by tide, wave, ocean'
    ],
    'Geothermal': [
        'electricity production, deep geothermal'
    ],
    'Nuclear': [
        'electricity production, nuclear, boiling water reactor',
        'electricity production, nuclear, pressure water reactor, heavy water moderated',
        'electricity production, nuclear, pressure water reactor',
    ],
    'Fossil Brown coal/Lignite': [
        'electricity production, lignite',
        'heat and power co-generation, lignite',
    ],
    'Fossil Hard coal': [
        'electricity production, hard coal',
        'electricity production, lignite',
        'electricity production, hard coal, conventional',
        'electricity production, hard coal, supercritical',
        'heat and power co-generation, hard coal',
    ],
    'Fossil Gas': [
        'electricity production, natural gas, 10MW',
        'electricity production, natural gas, combined cycle power plant',
        'electricity production, natural gas, conventional power plant',
        'natural gas, burned in micro gas turbine, 100kWe',
        'heat and power co-generation, natural gas, 160kW electrical, Jakobsberg',
        'heat and power co-generation, natural gas, 160kW electrical, lambda=1',
        'heat and power co-generation, natural gas, 1MW electrical, lean burn',
        'heat and power co-generation, natural gas, 200kW electrical, lean burn',
        'heat and power co-generation, natural gas, 500kW electrical, lean burn',
        'heat and power co-generation, natural gas, 50kW electrical, lean burn',
        'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
        'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
        'heat and power co-generation, natural gas, mini-plant 2KW electrical',
    ],
    'Hydro Pumped Storage': [
        'electricity production, hydro, pumped storage'
    ],
    'Fossil Oil': [
        'electricity production, oil',
         'heat and power co-generation, oil',
    ],
    'Fossil Peat': [
        'electricity production, peat'
    ],
    'Fossil Oil shale': [
        'burnt shale production'
    ],
    'Biomass': [
        'heat and power co-generation, biogas, gas engine',
        'heat and power co-generation, biogas, gas engine, label-certified',
        'heat and power co-generation, wood chips, 2000 kW',
        'heat and power co-generation, wood chips, 2000 kW, state-of-the-art 2014',
        'heat and power co-generation, wood chips, 6667 kW',
        'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
        'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014, label-certified',
        'wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future',
        'electricity production, wood, future',
        'biogas, burned in micro gas turbine 100kWe',
    ],
    'Fossil Coal-derived gas': [
        'treatment of coal gas, in power plant'
    ],
}